# Install object detection API

In [0]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
!git clone --quiet https://github.com/tensorflow/models.git
  
import os
os.chdir('models/research')
os.environ['PYTHONPATH'] += ':/content/models/research'
os.environ['PYTHONPATH'] += ':/content/models/research/slim'

!protoc object_detection/protos/*.proto --python_out=.

In [0]:
#%run object_detection/builders/model_builder_test.py
!python object_detection/builders/model_builder_test.py

# Prepare training data

In [0]:
# Replace "FILE_ID" with file id
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=FILE_ID" -O data.zip # && rm -rf /tmp/cookies.txt  
  
!unzip data.zip
!rm data.zip

In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

!tar -xzvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
!mkdir -p checkpoints
!mv faster_rcnn_inception_v2_coco_2018_01_28/ checkpoints/
!rm faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
!mkdir -p models/model
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1I5CyWK3UAfAdpc9A9Man0FImh0JDsc0J' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1I5CyWK3UAfAdpc9A9Man0FImh0JDsc0J" -O faster_rcnn_inception_v2_coco.config
!mv faster_rcnn_inception_v2_coco.config models/model/

# Train the model

In [0]:
!mkdir -p models/train

In [0]:
# Silence warnings and infos
!export TF_CPP_MIN_LOG_LEVEL=3

In [0]:
!python object_detection/train.py \
    --logtostderr \
    --pipeline_config_path models/model/faster_rcnn_inception_v2_coco.config \
    --train_dir models/train

# Export inference graph

In [0]:
!python object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path models/model/faster_rcnn_inception_v2_coco.config \
    --trained_checkpoint_prefix models/train/model.ckpt-100 \
    --output_directory models/eval/my_model_100

In [0]:
from google.colab import files

files.download('models/eval/my_model_100/frozen_inference_graph.pb')

# GPU info

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()